In [ ]:
# This cell is only for setting example parameter defaults - gets replaced by sidecar.
ra = 'all'
dec = 'all'
ra = '225.347166'
dec = '-39.272637'
time = 'now'
bands = 'all'

# What's the predicted skybrightness?

In [ ]:
import os
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord, AltAz, get_body
from astropy.time import Time
import astropy.units as u

from rubin_scheduler.scheduler.utils import generate_all_sky
from rubin_scheduler.utils import Site
from rubin_sim.skybrightness import SkyModel
import rubin_sim.phot_utils as phot_utils

import healpy as hp
import matplotlib.pyplot as plt

In [ ]:
if 'usdf' in os.getenv("EXTERNAL_INSTANCE_URL", ""):
    os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

In [ ]:
if time == 'now':
    obs_time = Time.now()
else:
    obs_time = Time(time, format='iso', scale='utc')

if ra == 'all' or dec == 'all':
    nside = 64
    sky = generate_all_sky(nside=nside, elevation_limit=20)
    ra_val = sky['ra']
    dec_val = sky['dec']
    all_sky = True
else:
    ra_val = float(ra)
    dec_val = float(dec)
    all_sky = False

if bands == 'all':
    bands = "ugrizy"
else:
    if isinstance(bands, str):
        if len(bands) == 1:
            bands = [bands]

In [ ]:
lsst_site = Site("LSST")
lsst_loc = lsst_site.to_earth_location()
skybg_model = SkyModel(mags=True)

lst = obs_time.sidereal_time('apparent', lsst_loc)

aa = AltAz(location = lsst_loc, obstime=obs_time)
skycoords = SkyCoord(ra=ra_val * u.deg, dec=dec_val * u.deg, frame="icrs")
altaz = skycoords.transform_to(aa)

sun_coords = get_body('sun', obs_time, lsst_loc)
moon_coords = get_body('moon', obs_time, lsst_loc)

print(f"Checking @ obs_time = {obs_time.iso}, LST {lst :.3f}")
if all_sky: 
    print(f"For the whole sky.")
else:
    print(f"For RA {ra_val} and Dec {dec_val}")
    
    print(f"This point is at an airmass of {altaz.secz :.2f}")

In [ ]:
# Calculate sky background.
#skybg_model.set_ra_dec_mjd(ra_val, dec_val, obs_time.mjd, degrees=True)

from rubin_scheduler.site_models import Almanac
almanac = Almanac()


skycoords = SkyCoord(ra=ra_val * u.deg, dec=dec_val * u.deg, frame="icrs")
times = np.arange(Time("2025-05-25T12:00:00", format='isot', scale='tai').mjd, Time("2025-06-10T12:00:00", format='isot', scale='tai').mjd, 0.5/24)
moon_illum = almanac.get_sun_moon_positions(times)['moon_phase']
sun_alt = almanac.get_sun_moon_positions(times)['sun_alt']
sky_mags = {}
for b in bands:
    sky_mags[b] = []
airmass = []
for t in times:
    tt = Time(t, format='mjd', scale='tai')
    lst = tt.sidereal_time('apparent', lsst_loc)
    aa = AltAz(location = lsst_loc, obstime=tt)
    altaz = skycoords.transform_to(aa)
    x = altaz.secz
    airmass.append(x)
    if x < 2.9 and x >= 1.0:
        skybg_model.set_ra_dec_mjd(ra_val, dec_val, t, degrees=True)
        vals = skybg_model.return_mags()
        for b in bands:
            sky_mags[b].append(vals[b][0])
    else:
        for b in bands:
            sky_mags[b].append(np.nan)

airmass = np.array(airmass)
for b in bands:
    sky_mags[b] = np.array(sky_mags[b])

In [ ]:
b = 'i'
new_moon = np.where((moon_illum < 3) & (sun_alt < np.radians(-12)))[0]
now = np.where((times - Time("2025-06-04T12:00:00", format='isot', scale='tai').mjd < 1) & (times - Time("2025-06-04T12:00:00", format='isot', scale='tai').mjd > 0) & (sun_alt < np.radians(-12)))[0]

plt.scatter((times[new_moon] - 0.5) % 1, sky_mags[b][new_moon], c=airmass[new_moon])
plt.plot((times[new_moon] - 0.5) % 1, sky_mags[b][new_moon], zorder=0)
plt.scatter((times[now]- 0.5) % 1, sky_mags[b][now], c=airmass[now], marker='s')
plt.plot((times[now]- 0.5) % 1, sky_mags[b][now], zorder=0)
plt.colorbar(label='airmass')
plt.gca().invert_yaxis()
plt.ylabel(f"{b} skybrightness (mag/arcsecond^2)")
plt.xlabel("Fractional dayobs")

In [ ]:
platescale = 0.2
exptimes = np.arange(14, 40, 2)
idx = np.where(exptimes == 30)[0]
sky_counts = {}
for b in 'ugrizy':
    sky_counts[b] = np.zeros((len(exptimes), len(sky_mags[b])))
    for i, exptime in enumerate(exptimes):
        zp_sky = phot_utils.predicted_zeropoint_hardware(b, exptime) 
        sky_counts[b][i, :] = np.power(10, (sky_mags[b] - zp_sky)/-2.5) * platescale**2

In [ ]:
if all_sky:
    rot = (lst.deg, lsst_site.latitude, 0)
    for b in bands:  
        hp.mollview(sky_mags[b], rot=rot, title=f"Sky Mags {b}")
        hp.graticule(dpar=30, dmer=60, alpha=0.3)
        hp.projscatter(moon_coords.ra, moon_coords.dec, color='b', label='Moon', lonlat=True)
        hp.projscatter(sun_coords.ra, sun_coords.dec, color='y', label='Sun', lonlat=True)
        _  = plt.legend(loc=(1.01, 0.5))
    print("Plot of background sky brightness in mag/arcsecond^2, RA/Dec grid, centered on location of Rubin")

In [ ]:
if all_sky:
    print("Describe sky_mags")
    sky_df = pd.DataFrame(sky_mags)
    display(sky_df.describe())
    print("Describe sky counts (30s exposure)")
    sky_df = pd.DataFrame(dict([(b, sky_counts[b][idx, :][0]) for b in 'ugrizy']))
    display(sky_df.describe())
else:
    sky_df = pd.concat([pd.DataFrame(sky_mags, index=["Mags"]), pd.DataFrame(dict([(b,sky_counts[b][idx, 0]) for b in 'ugrizy']), index=["Counts"])])
    display(sky_df.round(2))